In [1]:
import tensorflow as tf

---
---
---

# 同步处理数据

In [4]:
# 模拟一下同步先处理数据，然后才能取数据训练
# tensorflow当中，运行操作有依赖性

# 1、首先定义队列
Q = tf.FIFOQueue(3, tf.float32)

# 放入一些数据
enq_many = Q.enqueue_many([[0.1, 0.2, 0.3], ])

# 2、定义一些处理数据的螺距，取数据的过程      取数据，+1， 入队列

out_q = Q.dequeue()

data = out_q + 1

en_q = Q.enqueue(data)

with tf.Session() as sess:
    # 初始化队列
    sess.run(enq_many)

    # 处理数据
    for i in range(100):
        sess.run(en_q)

    # 训练数据
    for i in range(Q.size().eval()):
        print(sess.run(Q.dequeue()))

33.2
33.3
34.1


---
---
---

# 异步处理数据
## 线程协调器

In [9]:
# 模拟异步子线程 存入样本， 主线程 读取样本

# 1、定义一个队列，1000
Q = tf.FIFOQueue(1000, tf.float32)

# 2、定义要做的事情 循环 值，+1， 放入队列当中
var = tf.Variable(0.0)

# 实现一个自增  tf.assign_add
data = tf.assign_add(var, tf.constant(1.0))

en_q = Q.enqueue(data)

# 3、定义队列管理器op, 指定多少个子线程，子线程该干什么事情
qr = tf.train.QueueRunner(Q, enqueue_ops=[en_q] * 2)

# 初始化变量的OP
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    # 初始化变量
    sess.run(init_op)

    # 开启线程管理器
    coord = tf.train.Coordinator()

    # 真正开启子线程
    threads = qr.create_threads(sess, coord=coord, start=True)

    # 主线程，不断读取数据训练
    for i in range(300):
        print(sess.run(Q.dequeue()))

    # 回收你
    coord.request_stop()

    coord.join(threads)


# 批处理大小，跟队列，数据的数量没有影响，只决定 这批次取多少数据

5.0
48.0
95.0
127.0
144.0
187.0
212.0
251.0
299.0
336.0
376.0
391.0
415.0
447.0
475.0
509.0
548.0
580.0
619.0
655.0
688.0
709.0
743.0
761.0
778.0
818.0
850.0
874.0
910.0
930.0
974.0
1023.0
1024.0
1025.0
1026.0
1027.0
1028.0
1029.0
1030.0
1031.0
1032.0
1033.0
1034.0
1035.0
1036.0
1037.0
1038.0
1039.0
1040.0
1041.0
1042.0
1043.0
1044.0
1045.0
1046.0
1047.0
1048.0
1049.0
1050.0
1051.0
1052.0
1053.0
1054.0
1055.0
1056.0
1057.0
1058.0
1059.0
1060.0
1061.0
1062.0
1063.0
1064.0
1065.0
1066.0
1067.0
1068.0
1069.0
1070.0
1071.0
1072.0
1073.0
1074.0
1075.0
1076.0
1077.0
1078.0
1079.0
1080.0
1081.0
1082.0
1083.0
1084.0
1085.0
1086.0
1087.0
1088.0
1089.0
1090.0
1092.0
1092.0
1093.0
1094.0
1095.0
1096.0
1097.0
1098.0
1099.0
1100.0
1101.0
1102.0
1103.0
1104.0
1105.0
1106.0
1108.0
1108.0
1109.0
1110.0
1111.0
1112.0
1113.0
1114.0
1115.0
1116.0
1117.0
1118.0
1119.0
1120.0
1121.0
1122.0
1123.0
1124.0
1125.0
1126.0
1127.0
1128.0
1129.0
1130.0
1131.0
1132.0
1133.0
1135.0
1135.0
1136.0
1137.0
1138.0
1139.0

In [10]:
Q = tf.FIFOQueue(1000, tf.float32)

var = tf.Variable(0.0, tf.float32)

data = tf.assign_add(var, tf.constant(1.0))

en_q = Q.enqueue(data)

qr = tf.train.QueueRunner(Q, enqueue_ops=[en_q] * 2)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    
    threads = qr.create_threads(sess=sess,coord=coord, start=True)
    
    for i in range(300):
        print(sess.run(Q.dequeue()))
        
    coord.request_stop()
    
    coord.join(threads=threads)

5.0
53.0
81.0
115.0
140.0
177.0
201.0
244.0
278.0
299.0
326.0
376.0
401.0
456.0
477.0
504.0
540.0
581.0
619.0
635.0
658.0
677.0
719.0
771.0
805.0
830.0
861.0
901.0
946.0
981.0
1014.0
1023.0
1024.0
1025.0
1026.0
1027.0
1028.0
1029.0
1030.0
1031.0
1032.0
1033.0
1034.0
1035.0
1036.0
1037.0
1038.0
1039.0
1040.0
1041.0
1042.0
1043.0
1044.0
1045.0
1046.0
1047.0
1048.0
1049.0
1050.0
1051.0
1052.0
1053.0
1054.0
1055.0
1056.0
1057.0
1058.0
1059.0
1060.0
1061.0
1062.0
1064.0
1064.0
1065.0
1066.0
1067.0
1068.0
1069.0
1070.0
1071.0
1072.0
1073.0
1074.0
1075.0
1076.0
1077.0
1078.0
1080.0
1081.0
1081.0
1082.0
1083.0
1084.0
1085.0
1086.0
1087.0
1088.0
1089.0
1090.0
1091.0
1092.0
1093.0
1094.0
1095.0
1096.0
1097.0
1098.0
1099.0
1100.0
1101.0
1102.0
1103.0
1104.0
1105.0
1106.0
1107.0
1108.0
1109.0
1110.0
1111.0
1112.0
1113.0
1114.0
1115.0
1116.0
1117.0
1118.0
1119.0
1120.0
1121.0
1122.0
1123.0
1124.0
1125.0
1126.0
1127.0
1128.0
1129.0
1130.0
1131.0
1132.0
1133.0
1134.0
1135.0
1136.0
1137.0
1138.0
1139.

---
---
---

# 文件读取流程

In [9]:

#     读取CSV文件
#     :param filelist: 文件路径+名字的列表
#     :return: 读取的内容
filelist = ['./a.csv', './b.csv']

# 1、构造文件的队列
file_queue = tf.train.string_input_producer(filelist)

# 2、构造csv阅读器读取队列数据（按一行）
reader = tf.TextLineReader()

key, value = reader.read(file_queue)
# print(key, value)

# 3、对每行内容解码
# record_defaults:指定每一个样本的每一列的类型，指定默认值[["None"], [4.0]]
records = [["None"], [1.0]]

example, label = tf.decode_csv(value, record_defaults=records)

# 4、想要读取多个数据，就需要批处理
example_batch, label_batch = tf.train.batch([example, label], batch_size=12, num_threads=1, capacity=9)

print(example_batch, label_batch)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    
    threads = tf.train.start_queue_runners(sess, coord=coord)
    
    print(sess.run([example, label]))
    
    print(sess.run([example_batch, label_batch]))
    
    coord.request_stop()
    
    coord.join(threads=threads)

Tensor("batch_3:0", shape=(12,), dtype=string) Tensor("batch_3:1", shape=(12,), dtype=float32)
[b'a1', 1.0]
[array([b'a2', b'a3', b'b4', b'b5', b'b6', b'b4', b'b5', b'b6', b'a1',
       b'a2', b'a3', b'a1'], dtype=object), array([2., 3., 4., 5., 6., 4., 5., 6., 1., 2., 3., 1.], dtype=float32)]


---
---
---

# 图像处理

In [10]:
    
#     读取狗图片并转换成张量
#     :param filelist: 文件路径+ 名字的列表
#     :return: 每张图片的张量

filelist = ['.\9150e4e5gw1faae2uxkz6j20k00olq3j.jpg']

# 1、构造文件队列
file_queue = tf.train.string_input_producer(filelist)

# 2、构造阅读器去读取图片内容（默认读取一张图片）
reader = tf.WholeFileReader()

key, value = reader.read(file_queue)

print('value:', value)

# 3、对读取的图片数据进行解码
image = tf.image.decode_jpeg(value)

print('\nimage:', image)

# 5、处理图片的大小（统一大小）
image_resize = tf.image.resize_images(image, [200, 200])

print('\nimage_resize:', image_resize)

# 注意：一定要把样本的形状固定 [200, 200, 3],在批处理的时候要求所有数据形状必须定义
image_resize.set_shape([200, 200, 3])

print('\nimage_resize:', image_resize)

# 6、进行批处理
image_batch = tf.train.batch([image_resize], batch_size=20, num_threads=1, capacity=20)

print('\nimage_batch:', image_batch)

# with tf.Session() as sess:
#     coord = tf.train.Coordinator()
    
#     threads = tf.train.start_queue_runners(sess, coord=coord)
    
#     print(sess.run([example, label]))
    
#     print(sess.run([example_batch, label_batch]))
    
#     coord.request_stop()
    
#     coord.join(threads=threads)

value: Tensor("ReaderReadV2_4:1", shape=(), dtype=string)

image: Tensor("DecodeJpeg_1:0", shape=(?, ?, ?), dtype=uint8)

image_resize: Tensor("resize_images_1/Squeeze:0", shape=(200, 200, ?), dtype=float32)

image_resize: Tensor("resize_images_1/Squeeze:0", shape=(200, 200, 3), dtype=float32)

image_batch: Tensor("batch_4:0", shape=(20, 200, 200, 3), dtype=float32)


In [ ]:
tf.app.flags.DEFINE_string("cifar_dir", "./data/cifar10/cifar-10-batches-bin/", "文件的目录")
tf.app.flags.DEFINE_string("cifar_tfrecords", "./tmp/cifar.tfrecords", "存进tfrecords的文件")

FLAGS = tf.app.flags.FLAGS


class CifarRead(object):
    """完成读取二进制文件， 写进tfrecords，读取tfrecords
    """
    def __init__(self, filelist):
        # 文件列表
        self.file_list = filelist

        # 定义读取的图片的一些属性
        self.height = 32
        self.width = 32
        self.channel = 3
        # 二进制文件每张图片的字节
        self.label_bytes = 1
        self.image_bytes = self.height * self.width * self.channel
        self.bytes = self.label_bytes + self.image_bytes

    def read_and_decode(self):
        # 1、构造文件队列
        file_queue = tf.train.string_input_producer(self.file_list)

        # 2、构造二进制文件读取器，读取内容, 每个样本的字节数
        reader = tf.FixedLengthRecordReader(self.bytes)

        key, value = reader.read(file_queue)

        # 3、解码内容, 二进制文件内容的解码
        label_image = tf.decode_raw(value, tf.uint8)

        print(label_image)

        # 4、分割出图片和标签数据，切除特征值和目标值
        label = tf.cast(tf.slice(label_image, [0], [self.label_bytes]), tf.int32)

        image = tf.slice(label_image, [self.label_bytes], [self.image_bytes])

        # 5、可以对图片的特征数据进行形状的改变 [3072] --> [32, 32, 3]
        image_reshape = tf.reshape(image, [self.height, self.width, self.channel])

        print(label, image_reshape)
        # 6、批处理数据
        image_batch, label_batch = tf.train.batch([image_reshape, label], batch_size=10, num_threads=1, capacity=10)

        print(image_batch, label_batch)
        
        return image_batch, label_batch

    def write_ro_tfrecords(self, image_batch, label_batch):
        """
        将图片的特征值和目标值存进tfrecords
        :param image_batch: 10张图片的特征值
        :param label_batch: 10张图片的目标值
        :return: None
        """
        # 1、建立TFRecord存储器
        writer = tf.python_io.TFRecordWriter(FLAGS.cifar_tfrecords)

        # 2、循环将所有样本写入文件，每张图片样本都要构造example协议
        for i in range(10):
            # 取出第i个图片数据的特征值和目标值
            image = image_batch[i].eval().tostring()

            label = int(label_batch[i].eval()[0])

            # 构造一个样本的example
            example =  tf.train.Example(features=tf.train.Features(feature={
                "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
            }))

            # 写入单独的样本
            writer.write(example.SerializeToString())

        # 关闭
        writer.close()

    def read_from_tfrecords(self):
        # 1、构造文件队列
        file_queue = tf.train.string_input_producer([FLAGS.cifar_tfrecords])

        # 2、构造文件阅读器，读取内容example,value=一个样本的序列化example
        reader = tf.TFRecordReader()

        key, value = reader.read(file_queue)

        # 3、解析example
        features = tf.parse_single_example(value, features={
            "image": tf.FixedLenFeature([], tf.string),
            "label": tf.FixedLenFeature([], tf.int64),
        })

        # 4、解码内容, 如果读取的内容格式是string需要解码， 如果是int64,float32不需要解码
        image = tf.decode_raw(features["image"], tf.uint8)

        # 固定图片的形状，方便与批处理
        image_reshape = tf.reshape(image, [self.height, self.width, self.channel])

        label = tf.cast(features["label"], tf.int32)

        print(image_reshape, label)

        # 进行批处理
        image_batch, label_batch = tf.train.batch([image_reshape, label], batch_size=10, num_threads=1, capacity=10)

        return image_batch, label_batch
    

if __name__ == '__main__':
    cf = CifarRead(filelist)

    # image_batch, label_batch = cf.read_and_decode()

    image_batch, label_batch = cf.read_from_tfrecords()

    
    with tf.Session() as sess:
        coord = tf.train.Coordinator()

        threads = tf.train.start_queue_runners(sess, coord=coord)
        
        # 存进tfrecords文件
        # print("开始存储")
        #
        # cf.write_ro_tfrecords(image_batch, label_batch)
        #
        # print("结束存储")
        
        # 打印读取的内容
        print(sess.run([image_batch, label_batch]))

        coord.request_stop()

        coord.join(threads=threads)